In [3]:
# Import functions and libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy

from numpy import pi
from numpy import sin
from numpy import zeros
from numpy import r_
from scipy import signal
from scipy import misc # pip install Pillow
import matplotlib.pylab as pylab
import imageio

import matplotlib.pyplot as plt
import os 
import numpy as np
from PIL import Image
import scipy.io 

import numpy as np
from scipy.fftpack import dct, idct


#%matplotlib inline
#pylab.rcParams['figure.figsize'] = (20.0, 7.0)

In [4]:
# load my dataset
matPath = "/Volumes/cardiovascolare_phd/Deep_learning_python_proj_Data_augmentation/data_images_lymphoma/DatasColor_29.mat"
data = scipy.io.loadmat(matPath)
 
DIV = data['DATA'][0][2] # Division between training and test set
DIM1 = 299  # Number of training patterns
DIM2 = 374 # Number of patterns
NF = 5 
yE = data['DATA'][0][1]  # Labels of the patterns 
Images = data["DATA"][0][0][0] # Images

In [5]:
# another type of DCT implemetnation
def method1_DCT(im):
    """
    - "Discrte cosine transform" (DCT) method_1 function.
    - Apply the discrte cosine transform algo to the input image. 
    - (data points in terms of a sum of cosine functions oscillating at different frequencies)
    Params: 
    * img: input image. 
    """
    import numpy as np
    image = np.zeros_like(im, dtype=np.uint8)
    for i in range(3):
        # apply DCT to every dimension of the image
        DCT = dct(dct(im[:, :, i], axis=0, norm='ortho'), axis=1, norm='ortho')
        d1 = DCT.copy() # a copy for unmodified pixel in position 1. 
        # set some pixel to 0
        # changing the probability vector you can change the selection
        zero_indices = np.random.choice([False, True], size=DCT.shape, p=[0.4, 0.6]) # 0.6 probability of TRUE and 0.4 probability of FALSE. 
        #----------------------------------------------------------------------------
        # set all the pixels that have a value higher than (mode + std*2) or lower than (mode - std*2) to a value of zero
        values, counts = np.unique(im, return_counts=True)
        mode_value = values[np.argmax(counts)]
        std_dev = np.std(im)
        condition1 = np.abs(DCT) < (mode_value - std_dev*2)
        condition2 = np.abs(DCT) > (mode_value + std_dev*2)
        zero_indices = np.logical_or(condition1, condition2) # all pixels lower than threshold are setted to zero. 
        DCT[zero_indices] = 0
        #----------------------------------------------------------------------------        
        # leave unmodified pixel in position (1,1)
        DCT[0, 0] = d1[0, 0]
        # apply inverse DCT
        image[:, :, i] = idct(idct(DCT, axis=0, norm='ortho'), axis=1, norm='ortho')
    # return the image
    return image.astype(np.uint8)

In [6]:
def method2dct(im):
    y, x, z = im.shape
    
    # Apply standard deviation for every level
    sigma2 = np.std(im.astype(float))
    sigma1 = np.std(sigma2)
    sigma = np.std(sigma1) / 2
    
    image = np.zeros_like(im, dtype=np.uint8)
    
    for channel in range(z):
        # Apply DCT
        DCTim = dct(dct(im[:, :, channel], axis=0, norm='ortho'), axis=1, norm='ortho')
        
        for riga in range(y):
            for colonna in range(x):
                # DTC(1,1) unmodified
                if riga == 0 and colonna == 0:
                    continue
                else:
                    # Calculate random number between (-0.5, 0.5). Use np.random()
                    random_z = np.random.random()
                    while random_z > 0.5:
                        random_z = np.random.random()
                    
                    prob = np.random.random()
                    if prob > 0.5:
                        random_z *= 1 
                    else:
                        random_z *= -1
                    
                    # Modify DCT image
                    DCTim[riga, colonna] += sigma * random_z
            
        # Inverse DCT
        image[:, :, channel] = idct(idct(DCTim, axis=0, norm='ortho'), axis=1, norm='ortho')
    
    return image.astype(np.uint8)


In [11]:
def create_new_images_method(Images, directory_path,method): # method is an integer
    
    label_list = [] # List of labels for each image generated. 
    image_list = [] # List of all the np.arrays of images. 
    
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        print("\n-----------------------------------------------------")
        print(f"Directory '{directory_path}' created successfully.")
        print("New images will be create using method_1 or method_2")
        print("-----------------------------------------------------")

        
        # for each image of the list of images create a new image and save it into the folder
        if method == 1: 
            for i, img in enumerate(Images): 

                # take label and create the new image
                label = yE[0][i] # take the label of the image
                new_img = method1_DCT(img) #create the new image

                # add to the lists
                label_list.append(label)
                image_list.append(new_img)
                
                # just save the images as png
                # image_to_save = Image.fromarray(new_img)
                # image_to_save.save(f"{directory_path}/new_img_{i}.png")
            
            return label_list, image_list
    
        if method == 2:
            for i, img in enumerate(Images): 
                
                # take label and create the new image
                label = yE[0][i] # take the label of the image
                new_img = method2dct(img) #create the new image

                # add to the lists
                label_list.append(label)
                image_list.append(new_img)
                
                # save image as .png
                # image_to_save = Image.fromarray(new_img)
                # image_to_save.save(f"{directory_path}/new_img_{i}.png")
                
            return label_list, image_list
        
        if method < 1 or method > 2:
            print("\n------------------------------------------------------------------")
            print("------ Method not available. (method must be: 1<= method =>2) ------")
            print("\n------------------------------------------------------------------")
        
    else:
        print("\n---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
        print(f"Directory '{directory_path}' already exists!.")
        print("New images will be putted inside using method_1 or method_2 if the directory is empty.")
        print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n")
        
        # Check if the directory is empty
        is_empty = len(os.listdir(directory_path)) == 0
        
        if is_empty:
            # for each image of the list of images create a new image and save it into the folder
            if method == 1:
                for i, img in enumerate(Images): 

                    # take label and create the new image
                    label = yE[0][i] # take the label of the image
                    new_img = method1_DCT(img) #create the new image

                    # add to the lists
                    label_list.append(label)
                    image_list.append(new_img)

                    # just save the images as png
                    # image_to_save = Image.fromarray(new_img)
                    # image_to_save.save(f"{directory_path}/new_img_{i}.png")

                return label_list, image_list
            
            if method == 2:
                for i, img in enumerate(Images): 
                    
                    # take label and create the new image
                    label = yE[0][i] # take the label of the image
                    new_img = method2dct(img) #create the new image

                    # add to the lists
                    label_list.append(label)
                    image_list.append(new_img)

                    # save the image as .png 
                    # image_to_save = Image.fromarray(new_img)
                    # image_to_save.save(f"{directory_path}/new_img_{i}.png") 
                
                return label_list, image_list

        else:
            print("--- Directory is not empty. I will not insert new images ---\n ")
            if method == 1:
                for i, img in enumerate(Images): 

                    # take label and create the new image
                    label = yE[0][i] # take the label of the image
                    new_img = method1_DCT(img) #create the new image

                    # add to the lists
                    label_list.append(label)
                    image_list.append(new_img)

                    # just save the images as png
                    # image_to_save = Image.fromarray(new_img)
                    # image_to_save.save(f"{directory_path}/new_img_{i}.png")

                return label_list, image_list
            
            if method == 2:
                for i, img in enumerate(Images): 
                    
                    # take label and create the new image
                    label = yE[0][i] # take the label of the image
                    new_img = method2dct(img) #create the new image

                    # add to the lists
                    label_list.append(label)
                    image_list.append(new_img)

                    # save the image as .png 
                    # image_to_save = Image.fromarray(new_img)
                    # image_to_save.save(f"{directory_path}/new_img_{i}.png") 
                
                return label_list, image_list


In [12]:
# Create DCT images method 1 for all of the images in you dataset:
path_to_load = "/Volumes/cardiovascolare_phd/Deep_learning_python_proj_Data_augmentation/data_images_lymphoma/Method_1_images_data_augment"
label_list_m1, image_list_m1 = create_new_images_method(Images,path_to_load,1)


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Directory '/Volumes/cardiovascolare_phd/Deep_learning_python_proj_Data_augmentation/data_images_lymphoma/Method_1_images_data_augment' already exists!.
New images will be putted inside using method_1 if the directory is empty.
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

--- Directory is not empty. I will not insert new images ---
 


In [14]:
# Create new images using method 2 for the images of the dataset. 
path_to_load_m2 = "/Volumes/cardiovascolare_phd/Deep_learning_python_proj_Data_augmentation/data_images_lymphoma/Method_2_images_data_augment"
label_list_m2, image_list_m2 = create_new_images_method(Images,path_to_load_m2,2)


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Directory '/Volumes/cardiovascolare_phd/Deep_learning_python_proj_Data_augmentation/data_images_lymphoma/Method_2_images_data_augment' already exists!.
New images will be putted inside using method_1 if the directory is empty.
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

--- Directory is not empty. I will not insert new images ---
 


In [16]:
# --- CREATE THE AUGMENTED DATASET IMAGES ONLY ---
# Create a .mat file to generate a new .mat dataset with augmented images for method_1 and method_2
data_dict_m1_only_aug = {
    'images': image_list_m1,
    'labels': label_list_m1
}

data_dict_m2_only_aug = {
    'images': image_list_m2,
    'labels': label_list_m2
}

# Save as .mat file for method 1 and method 2. 
#For method_1
file_path_m1_only_aug = '/Volumes/cardiovascolare_phd/Deep_learning_python_proj_Data_augmentation/data_images_lymphoma/data_augmented_m1_only_aug.mat'
scipy.io.savemat(file_path_m1_only_aug, data_dict_m1_only_aug)

# For method_2
file_path_m2_only_aug = '/Volumes/cardiovascolare_phd/Deep_learning_python_proj_Data_augmentation/data_images_lymphoma/data_augmented_m2_only_aug.mat'
scipy.io.savemat(file_path_m2_only_aug, data_dict_m2_only_aug)

In [2]:
# Covert Imgaes and labels to lists
# --- CREATE DATASET OF ALL ORIGINAL IMAGES + THE AUGMENTED TYPE ONE ---
# Create a .mat file to generate a new .mat dataset with augmented images for method_1 and method_2
image_list_m1_tot = Images.tolist() + image_list_m1
label_list_m1_tot = yE[0].tolist() + label_list_m1
data_dict_m1 = {
    'images': image_list_m1_tot,
    'labels': label_list_m1_tot
}

image_list_m2_tot = Images_list + image_list_m2
label_list_m2_tot = ye_list + label_list_m2
data_dict_m2 = {
    'images': image_list_m2_tot,
    'labels': label_list_m2_tot
}

# Save as .mat file for method 1 and method 2. 
#For method_1
file_path_m1 = '.../data_augmented_m1.mat'
scipy.io.savemat(file_path_m1, data_dict_m1)

# For method_2
file_path_m2 = '.../data_augmented_m2.mat'
scipy.io.savemat(file_path_m2, data_dict_m2)


NameError: name 'Images' is not defined